# Check and explore model

In [1]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns
%matplotlib inline

sys.path.insert(0, (Path(".").resolve().parent / "tensorpack-FasterRCNN").as_posix())

In [2]:
from data import get_all_anchors, get_all_anchors_fpn
from train import ResNetFPNModel

from model_box import (
    clip_boxes, decode_bbox_target, encode_bbox_target,
    crop_and_resize, roi_align, RPNAnchors)

from data import cfg

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
model = ResNetFPNModel()

inputs = model.inputs()

In [4]:
inputs

[<tf.Tensor 'image:0' shape=(?, ?, 3) dtype=float32>,
 <tf.Tensor 'anchor_labels_lvl2:0' shape=(?, ?, 3) dtype=int32>,
 <tf.Tensor 'anchor_boxes_lvl2:0' shape=(?, ?, 3, 4) dtype=float32>,
 <tf.Tensor 'anchor_labels_lvl3:0' shape=(?, ?, 3) dtype=int32>,
 <tf.Tensor 'anchor_boxes_lvl3:0' shape=(?, ?, 3, 4) dtype=float32>,
 <tf.Tensor 'anchor_labels_lvl4:0' shape=(?, ?, 3) dtype=int32>,
 <tf.Tensor 'anchor_boxes_lvl4:0' shape=(?, ?, 3, 4) dtype=float32>,
 <tf.Tensor 'anchor_labels_lvl5:0' shape=(?, ?, 3) dtype=int32>,
 <tf.Tensor 'anchor_boxes_lvl5:0' shape=(?, ?, 3, 4) dtype=float32>,
 <tf.Tensor 'anchor_labels_lvl6:0' shape=(?, ?, 3) dtype=int32>,
 <tf.Tensor 'anchor_boxes_lvl6:0' shape=(?, ?, 3, 4) dtype=float32>,
 <tf.Tensor 'gt_boxes:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'gt_labels:0' shape=(?,) dtype=int64>,
 <tf.Tensor 'gt_masks:0' shape=(?, ?, ?) dtype=uint8>]

In [5]:
num_fpn_level = len(cfg.FPN.ANCHOR_STRIDES)
assert len(cfg.RPN.ANCHOR_SIZES) == num_fpn_level

image = inputs[0]
input_anchors = inputs[1: 1 + 2 * num_fpn_level]

In [6]:
all_anchors_fpn = get_all_anchors_fpn()
print(type(all_anchors_fpn), len(all_anchors_fpn))
for i, single_level_anchors in enumerate(all_anchors_fpn):
    print("Level {} | shape {}".format(i + 2, single_level_anchors.shape))

<class 'list'> 5
Level 2 | shape (334, 334, 3, 4)
Level 3 | shape (167, 167, 3, 4)
Level 4 | shape (84, 84, 3, 4)
Level 5 | shape (42, 42, 3, 4)
Level 6 | shape (21, 21, 3, 4)


In [7]:
multilevel_anchors = [RPNAnchors(*args) for args in
                      zip(get_all_anchors_fpn(), input_anchors[0::2], input_anchors[1::2])]

In [8]:
type(multilevel_anchors), len(multilevel_anchors)

(list, 5)

In [9]:
for i, rpn_anchors in enumerate(multilevel_anchors):
    print("Level {} | {}".format(i + 2, type(rpn_anchors)))

Level 2 | <class 'model_box.RPNAnchors'>
Level 3 | <class 'model_box.RPNAnchors'>
Level 4 | <class 'model_box.RPNAnchors'>
Level 5 | <class 'model_box.RPNAnchors'>
Level 6 | <class 'model_box.RPNAnchors'>


In [10]:
gt_boxes, gt_labels = inputs[11], inputs[12]

In [11]:
image = model.preprocess(image)

In [12]:
from basemodel import (
    image_preprocess, resnet_c4_backbone, resnet_conv5,
    resnet_fpn_backbone)

In [13]:
from config import finalize_configs

finalize_configs(cfg)

[0708 21:56:43 @config.py:191] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCK': [3, 4, 6, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': True,
              'WEIGHTS': ''},
 'DATA': {'BASEDIR': '.',
          'CLASS_NAMES': [],
          'NUM_CATEGORY': 601,
          'NUM_CLASS': 602,
          'TRAIN': ['test', 'val_0.7'],
          'VAL': 'val_0.3'},
 'FPN': {'ANCHOR_STRIDES': (4, 8, 16, 32, 64),
         'FRCNN_CONV_HEAD_DIM': 256,
         'FRCNN_FC_HEAD_DIM': 1024,
         'FRCNN_HEAD_FUNC': 'fastrcnn_2fc_head',
         'NUM_CHANNEL': 256,
         'RESOLUTION_REQUIREMENT': 32},
 'FRCNN': {'BATCH_PER_IM': 512,
           'BBOX_REG_WEIGHTS': [10.0, 10.0, 5.0, 5.0],
           'FG_RATIO': 0.25,
           'FG_THRESH': 0.5},
 'MODE_FPN': False,
 'MODE_MASK': True,
 'MRCNN': {'HEAD_DIM': 256},
 'PREPROC': {'MAX_SIZE': 1333,
             'PIXEL_MEAN': [123

In [14]:
from tensorpack.tfutils.tower import TowerContext

with TowerContext('tower0', is_training=True):
    c2345 = resnet_fpn_backbone(image, cfg.BACKBONE.RESNET_NUM_BLOCK)

[0708 21:56:43 @registry.py:121] conv0 input: [1, 3, None, None]
[0708 21:56:43 @batch_norm.py:164] WRN [BatchNorm] Using moving_mean/moving_variance in training.
[0708 21:56:43 @registry.py:129] conv0 output: [1, 64, None, None]
[0708 21:56:43 @registry.py:121] pool0 input: [1, 64, None, None]
[0708 21:56:43 @registry.py:129] pool0 output: [1, 64, None, None]
[0708 21:56:43 @registry.py:121] group0/block0/conv1 input: [1, 64, None, None]
[0708 21:56:43 @batch_norm.py:164] WRN [BatchNorm] Using moving_mean/moving_variance in training.
[0708 21:56:43 @registry.py:129] group0/block0/conv1 output: [1, 64, None, None]
[0708 21:56:43 @registry.py:121] group0/block0/conv2 input: [1, 64, None, None]
[0708 21:56:43 @batch_norm.py:164] WRN [BatchNorm] Using moving_mean/moving_variance in training.
[0708 21:56:43 @registry.py:129] group0/block0/conv2 output: [1, 64, None, None]
[0708 21:56:43 @registry.py:121] group0/block0/conv3 input: [1, 64, None, None]
[0708 21:56:43 @batch_norm.py:164] WRN 

In [15]:
c2345

(<tf.Tensor 'tower0/StopGradient:0' shape=(1, 256, ?, ?) dtype=float32>,
 <tf.Tensor 'tower0/group1/block3/output:0' shape=(1, 512, ?, ?) dtype=float32>,
 <tf.Tensor 'tower0/group2/block5/output:0' shape=(1, 1024, ?, ?) dtype=float32>,
 <tf.Tensor 'tower0/group3/block2/output:0' shape=(1, 2048, ?, ?) dtype=float32>)

In [16]:
from model_fpn import (
    fpn_model, multilevel_roi_align,
    multilevel_rpn_losses, generate_fpn_proposals)

In [17]:
fpn_model??

Signature: fpn_model(features)
Source:   
@layer_register(log_shape=True)
def fpn_model(features):
    """
    Args:
        features ([tf.Tensor]): ResNet features c2-c5

    Returns:
        [tf.Tensor]: FPN features p2-p6
    """
    assert len(features) == 4, features
    num_channel = cfg.FPN.NUM_CHANNEL

    def upsample2x(name, x):
        return FixedUnPooling(
            name, x, 2, unpool_mat=np.ones((2, 2), dtype='float32'),
            data_format='channels_first')

        # tf.image.resize is, again, not aligned.
        # with tf.name_scope(name):
        #     shape2d = tf.shape(x)[2:]
        #     x = tf.transpose(x, [0, 2, 3, 1])
        #     x = tf.image.resize_nearest_neighbor(x, shape2d * 2, align_corners=True)
        #     x = tf.transpose(x, [0, 3, 1, 2])
        #     return x

    with argscope(Conv2D, data_format='channels_first',
                  activation=tf.identity, use_bias=True,
                  kernel_initializer=tf.variance_scaling_initializer(s

In [19]:
import tensorflow as tf

features = [
    tf.placeholder(tf.float32, shape=(1, 256, None, None), name='c2'),
    tf.placeholder(tf.float32, shape=(1, 512, None, None), name='c3'),
    tf.placeholder(tf.float32, shape=(1, 1024, None, None), name='c4'),
    tf.placeholder(tf.float32, shape=(1, 2048, None, None), name='c5'),
]

In [20]:
p23456 = fpn_model(features)

IndexError: tuple index out of range